## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
from config import g_key
from config import weather_api_key

gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Discription
0,0,Punta Arenas,CL,-53.15,-70.92,42.80,81,100,19.46,overcast clouds
1,1,Hithadhoo,MV,-0.60,73.08,83.53,73,16,9.64,light rain
2,2,Clyde River,CA,70.47,-68.59,44.67,88,98,4.79,overcast clouds
3,3,Kaeo,NZ,-35.10,173.78,62.01,3,93,5.99,overcast clouds
4,4,Qaanaaq,GL,77.48,-69.36,35.47,84,100,0.65,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Discription
13,13,Atuona,PF,-9.80,-139.03,78.75,76,9,13.09,light rain
15,15,Luderitz,NaN,-26.65,15.16,70.29,20,0,24.27,clear sky
19,19,Albany,US,42.60,-73.97,71.01,88,87,3.29,overcast clouds
20,20,Avarua,CK,-21.21,-159.78,75.20,83,10,16.11,clear sky
22,22,Avera,US,33.19,-82.53,73.40,78,1,4.70,clear sky
23,23,Bambous Virieux,MU,-20.34,57.76,73.40,64,20,8.05,few clouds
27,27,Fare,PF,-16.70,-151.02,77.45,79,0,23.76,light rain
32,32,Saint-Pierre,RE,-21.34,55.48,71.60,46,0,4.70,clear sky
54,54,Souillac,MU,-20.52,57.52,73.40,64,20,8.05,few clouds
57,57,Carbonia,IT,39.17,8.52,77.00,83,40,2.24,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        171
City           171
Country        170
Lat            171
Lng            171
Max Temp       171
Humidity       171
Cloudiness     171
Wind Speed     171
Discription    171
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Discription", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Discription,Lat,Lng,Hotel Name
13,Atuona,PF,78.75,light rain,-9.80,-139.03,
15,Luderitz,NaN,70.29,clear sky,-26.65,15.16,
19,Albany,US,71.01,overcast clouds,42.60,-73.97,
20,Avarua,CK,75.20,clear sky,-21.21,-159.78,
22,Avera,US,73.40,clear sky,33.19,-82.53,
23,Bambous Virieux,MU,73.40,few clouds,-20.34,57.76,
27,Fare,PF,77.45,light rain,-16.70,-151.02,
32,Saint-Pierre,RE,71.60,clear sky,-21.34,55.48,
54,Souillac,MU,73.40,few clouds,-20.52,57.52,
57,Carbonia,IT,77.00,scattered clouds,39.17,8.52,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.

hotel_df["Hotel Name"].replace('', np.nan, inplace=True)

In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))